In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from efficient_apriori import apriori

# Read recipe inputs
online_Retail = dataiku.Dataset("Online_Retail_Customers")
df = online_Retail.get_dataframe()

In [0]:
df.info()

In [0]:
print(f'{len(df)} purchases with a customer id')

In [0]:
# grps = online_Retail_df.groupby('InvoiceNo')
grps = df.groupby('CustomerID')

In [0]:
invoice_products = []
customer_products = []
for key, item in grps:
    prods = item['StockCode'].str.cat(sep=',').split(',')

    if len(prods) > 1:
        products = tuple(np.unique(prods))
        invoice_products.append(products)

        customer_products.append({'CustomerID': key, 'Products': prods})
#     print(f'{key}: {cust_grps.get_group(key)["StockCode"]}')

In [0]:
# custom_variables = dataiku.get_custom_variables()
project_variables = dataiku.api_client().get_default_project().get_variables()

In [0]:
print(f'Generating rules for {project_variables["standard"]["Country"]}')

In [0]:
itemsets, rules = apriori(invoice_products, min_support=float(project_variables['standard']['min_support']),
                          min_confidence=float(project_variables['standard']['min_confidence']))

In [0]:
good_rule_filter = filter(lambda rule: len(rule.lhs) >= 2 and len(rule.rhs) == 1, rules)
good_rules_sorted = sorted(good_rule_filter, key=lambda rule: rule.lift)

In [0]:
good_rules = []
for rule in good_rules_sorted: ##sorted(good_rule_filter, key=lambda rule: rule.lift):
    good_rules.append({ "rule_lhs": rule.lhs, "rule_rhs": rule.rhs, "lift": rule.lift})

rule_set = dataiku.Dataset("rules")
rule_set.write_with_schema(pd.DataFrame.from_dict(good_rules))

In [0]:
recommendations = []
for cp in customer_products:
    for rule in good_rules_sorted:
        lhs_intersection = np.intersect1d(rule.lhs, cp['Products']) # do they have all the products in the rule?
        rhs_intersection = np.intersect1d(rule.rhs, cp['Products']) # do they already have the suggestion?

        if len(lhs_intersection) == len(rule.lhs) and len(rhs_intersection) == 0:
#             print(lhs_intersection)
#             print(f'recommending {rule.rhs[0]} to {cp["CustomerID"]}')
            rec = {'CustomerID': cp['CustomerID'], 'StockCode': rule.rhs[0]}

            ct = 1
            for r in rule.lhs:
                rec[f'LHS_{ct}'] = r
                ct = ct + 1

            recommendations.append(rec)

In [0]:
# Write recipe outputs
product_recommendations = dataiku.Dataset("product_recommendations")
product_recommendations.write_with_schema(pd.DataFrame.from_dict(recommendations))